In [1]:
import pandas as pd

In [2]:
geo = pd.read_csv('../../data/geo/geography_lookup.csv')

In [3]:
codes = geo.columns[geo.columns.str.endswith('CD') & ~geo.columns.isin(['WD22CD'])]
names = geo.columns[geo.columns.str.endswith('NM')]

In [4]:
def make_lookup(pair):
    pair = pd.Index(pair)
    type = pair[0][:-2]
    frame = geo.loc[:, pair].drop_duplicates().rename(columns=lambda c: c[-2:])
    frame['type'] = type
    return frame
geo_lookup = pd.concat([make_lookup(pair) for pair in zip(codes.to_list(), names.to_list())]).dropna()
geo_lookup.columns = ['code', 'name', 'type']
geo_lookup = geo_lookup.set_index('code')

In [5]:
place_data = pd.DataFrame()

for code in codes.to_series().iloc[::-1]:
    place = geo.loc[:, [code]]
    code_loc = codes.get_loc(code)
    child_codes = codes.to_series().iloc[code_loc+1:].values
    place_data = pd.concat([
        place_data,
        pd.DataFrame({
            'key': place[code],
            'children': geo.loc[:, child_codes].to_numpy().tolist(),
        }),])

place_data = place_data.explode('children').dropna().groupby('key').agg({
    'children': lambda x: x.drop_duplicates().sort_values().tolist(),
})
place_data = place_data.merge(right=geo_lookup, left_index=True, right_index=True, how='outer')
place_data.children[place_data.children.isna()] = place_data.children[place_data.children.isna()].apply(lambda x: [])

In [6]:
place_data.to_json(f'../../src/_data/places.json', orient='index', indent=2)

In [7]:
place_data

,children,name,type
E06000001,[],Hartlepool,LAD22
E06000002,[],Middlesbrough,LAD22
E06000003,[],Redcar and Cleveland,LAD22
E06000004,[],Stockton-on-Tees,LAD22
E06000005,[],Darlington,LAD22
...,...,...,...
E47000003,"[E08000032, E08000033, E08000034, E08000035, E...",West Yorkshire,CAUTH22
E47000004,"[E06000006, E08000011, E08000012, E08000013, E...",Liverpool City Region,CAUTH22
E47000006,"[E06000001, E06000002, E06000003, E06000004, E...",Tees Valley,CAUTH22
E47000010,"[E06000047, E08000023, E08000024, E08000037]",North East,CAUTH22
